In [ ]:
import face_recognition
import requests
from PIL import Image, ExifTags
from io import BytesIO
from IPython.display import display

# ฟังก์ชันสำหรับปรับการหมุนของรูปภาพตาม EXIF
def auto_orient_image(img):
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break
        exif = img._getexif()
        if exif is not None:
            orientation_value = exif.get(orientation, None)
            if orientation_value == 3:
                img = img.rotate(180, expand=True)
            elif orientation_value == 6:
                img = img.rotate(270, expand=True)
            elif orientation_value == 8:
                img = img.rotate(90, expand=True)
    except Exception as e:
        pass
    return img

# ฟังก์ชันโหลดรูปจาก URL
def load_image_from_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return face_recognition.load_image_file(BytesIO(response.content)), img

# ฟังก์ชันเปรียบเทียบใบหน้า
def is_same_person(url1, url2, threshold=0.5):
    img1_array, img1_pil = load_image_from_url(url1)
    img2_array, img2_pil = load_image_from_url(url2)

    print("🔍 ภาพที่ 1:")
    display(img1_pil)
    print("🔍 ภาพที่ 2:")
    display(img2_pil)

    encodings1 = face_recognition.face_encodings(img1_array)
    encodings2 = face_recognition.face_encodings(img2_array)
    face_locations2 = face_recognition.face_locations(img2_array)

     # ถ้าไม่พบใบหน้าในภาพที่ 2 เลย
    if not face_locations2:
        # print("❌ ไม่สามารถตรวจจับใบหน้าใดๆ ในภาพที่ 2 ได้")
        # display(img2_pil)
        return "❌ ไม่สามารถตรวจจับใบหน้าใดๆ ในภาพที่ 2 ได้"

    if not encodings1 :
        return "❌ ไม่พบใบหน้าในภาพที่ 1"
    if not encodings2:
        return "❌ ไม่พบใบหน้าในภาพที่ "    

    # distance = face_recognition.face_distance([encodings1[0]], encodings2[0])[0]

    # if distance < threshold:
    #     return f"✅ ใกล้เคียง: อาจเป็นคนเดียวกัน (distance = {distance:.2f})"
    # else:
    #     return f"❌ ไม่ตรงกัน: คนละคน (distance = {distance:.2f})"

    distances = face_recognition.face_distance(encodings2, encodings1[0])

   #for loop zoom multiple face 
    for idx, (face_loc, dist) in enumerate(zip(face_locations2, distances)):
            top, right, bottom, left = face_loc
            face_img = img2_pil.crop((left, top, right, bottom))
            print(f"🔎 คนที่ {idx+1} | distance = {dist:.2f}")
            display(face_img)

    min_distance = distances.min()
    min_index = distances.argmin()

    #zoom face 
    # if face_locations2 and min_index < len(face_locations2):
    #         top, right, bottom, left = face_locations2[min_index]
    #         face_img = img2_pil.crop((left, top, right, bottom))
    #         print("🔎 ใบหน้าที่ใกล้เคียงที่สุดในภาพที่ 2:")
    #         display(face_img)    

    if min_distance < threshold:
        return f"✅ พบใบหน้าที่ใกล้เคียงที่สุด (คนที่ {min_index+1}) (distance = {min_distance:.2f})"
    else:
        return f"❌ ไม่มีใบหน้าไหนตรงกับ default (min distance = {min_distance:.2f})"

# รูปพนักงาน
# url_default = "https://wms.advanceagro.net/WSVIS/api/Face/GetImage?CardID=688086"
url_default = "https://api-passport.advanceagro.net/qrapi/driver/57_d.jpg"

# url_new = "https://drive.google.com/uc?export=download&id="
#----------------------------------example for cannot detect face ----------------------------------

# url_new = "https://drive.google.com/uc?export=download&id=1doQz85QDGwWO1lSfscVEZB1WCGKZahJz"
# url_new = "https://drive.google.com/uc?export=download&id=1djZWka58x4sUX1uEJz2T_RB53Xj6k1jd"

#mask
# url_new = "https://drive.google.com/uc?export=download&id=1oDlPFtFXdgQ8nyGGSi1K40Sn6AiuNIwJ"

#----------------------------------example for detect successfully----------------------------------

# can detect but not same person 
# url_new = "https://drive.google.com/uc?export=download&id=1CWEzTjtH8ZjCdt80iujaJfrmRKpt7HyB"
# url_new = "https://wms.advanceagro.net/WSVIS/api/Face/GetImage?CardID=678069"

#1people 
# url_new = "https://drive.google.com/uc?export=download&id=16RpgIYZ1teAtJ2Ol4ZiaondmiAbQ6ryb"
# url_new = "https://drive.google.com/uc?export=download&id=1_cznadDWIyjdDvnahNsp--5UynDVuaxj"

#2people 
# url_new = "https://drive.google.com/uc?export=download&id=12N4vlF3shvRNMeyzW11LsCjXL0kyAoXA"
# url_new = "https://drive.google.com/uc?export=download&id=1vjZwPbo8pZqe9-9RMgoyJTXIwrPU-ZBE"

# ถูกคนแต่ detect ว่าไม่ได้
# url_new = "https://drive.google.com/uc?export=download&id=124bTxw7wv9Nj5GwVsHyWbxptt0O1pPga"
# url_new = "https://drive.google.com/uc?export=download&id=1ey2DzQ9SHetzoKyY8eJny4vXevNgzUGA" 

#3people
# url_new = "https://drive.google.com/uc?export=download&id=1U_ZCNcb8W-Y_e-qAv3lI1E6ix4vQF17a"

#group
# url_new = "https://drive.google.com/uc?export=download&id=1DIstBXIQ5TKy7um2T7-JBTP2Wk1MJtcl"

# Aws
url_new =    "https://api-passport.advanceagro.net/awsapi/filename/1749633011092_20250611_3410400683009.jpg"
# ✅ เปรียบเทียบและแสดงผล
result = is_same_person(url_default, url_new)
print(result)

In [ ]:
# ติดตั้ง mtcnn และ opencv-python
# %pip install mtcnn opencv-python
# %pip install tensorflow mtcnn opencv-python
# %pip install matplotlib
%pip install torch facenet_pytorch

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import requests
from io import BytesIO
import torch
from IPython.display import display
import numpy as np

# โหลดภาพจาก URL
def load_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert('RGB')

# โหลด MTCNN และ Resnet
mtcnn = MTCNN(image_size=160, margin=20)
resnet = InceptionResnetV1(pretrained='vggface2').eval()

# เปลี่ยน PIL เป็น embedding
def get_face_embedding(pil_image):
    face = mtcnn(pil_image)
    if face is None:
        return None
    with torch.no_grad():
        embedding = resnet(face.unsqueeze(0))  # ใส่ batch dim
    return embedding

# เปรียบเทียบใบหน้าด้วย Euclidean distance
def compare_faces_mtcnn_facenet(url1, url2, threshold=0.9):
    img1 = load_image(url1)
    img2 = load_image(url2)

    print("📸 ภาพที่ 1:")
    display(img1)
    print("📸 ภาพที่ 2:")
    display(img2)

    emb1 = get_face_embedding(img1)
    emb2 = get_face_embedding(img2)

    if emb1 is None:
        return "❌ ไม่พบใบหน้าในภาพที่ 1"
    if emb2 is None:
        return "❌ ไม่พบใบหน้าในภาพที่ 2"

    # Euclidean distance
    distance = (emb1 - emb2).norm().item()
    print(f"🔎 ความห่าง (Euclidean distance) = {distance:.4f}")

    if distance < threshold:
        return f"✅ ใกล้เคียง (ระยะห่าง = {distance:.4f})"
    else:
        return f"❌ คนละคน (ระยะห่าง = {distance:.4f})"
        
# รูปพนักงาน
url_default = "https://wms.advanceagro.net/WSVIS/api/Face/GetImage?CardID=688086"

# url_new = "https://drive.google.com/uc?export=download&id="
#----------------------------------example for cannot detect face ----------------------------------

# url_new = "https://drive.google.com/uc?export=download&id=1doQz85QDGwWO1lSfscVEZB1WCGKZahJz"
url_new = "https://drive.google.com/uc?export=download&id=1djZWka58x4sUX1uEJz2T_RB53Xj6k1jd"
#mask
# url_new = "https://drive.google.com/uc?export=download&id=1oDlPFtFXdgQ8nyGGSi1K40Sn6AiuNIwJ"

#----------------------------------example for detect successfully----------------------------------

# can detect but not same person 
# url_new = "https://drive.google.com/uc?export=download&id=1CWEzTjtH8ZjCdt80iujaJfrmRKpt7HyB"
# url_new = "https://wms.advanceagro.net/WSVIS/api/Face/GetImage?CardID=678069"

#1people 
# url_new = "https://drive.google.com/uc?export=download&id=16RpgIYZ1teAtJ2Ol4ZiaondmiAbQ6ryb"
# url_new = "https://drive.google.com/uc?export=download&id=1_cznadDWIyjdDvnahNsp--5UynDVuaxj"

#2people 
# url_new = "https://drive.google.com/uc?export=download&id=12N4vlF3shvRNMeyzW11LsCjXL0kyAoXA"
# url_new = "https://drive.google.com/uc?export=download&id=1vjZwPbo8pZqe9-9RMgoyJTXIwrPU-ZBE"

# ถูกคนแต่ detect ว่าไม่ได้
# url_new = "https://drive.google.com/uc?export=download&id=124bTxw7wv9Nj5GwVsHyWbxptt0O1pPga"
# url_new = "https://drive.google.com/uc?export=download&id=1ey2DzQ9SHetzoKyY8eJny4vXevNgzUGA" 

#3people
# url_new = "https://drive.google.com/uc?export=download&id=1U_ZCNcb8W-Y_e-qAv3lI1E6ix4vQF17a"

#group
# url_new = "https://drive.google.com/uc?export=download&id=1DIstBXIQ5TKy7um2T7-JBTP2Wk1MJtcl"
result = compare_faces_mtcnn_facenet(url_default, url_new)
print(result)    


In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image, ExifTags
import requests
from io import BytesIO
import torch
from IPython.display import display
import numpy as np


def auto_orient_image(img):
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == "Orientation":
                break
        exif = img._getexif()
        if exif is not None:
            orientation_value = exif.get(orientation, None)
            if orientation_value == 3:
                img = img.rotate(180, expand=True)
            elif orientation_value == 6:
                img = img.rotate(270, expand=True)
            elif orientation_value == 8:
                img = img.rotate(90, expand=True)
    except Exception as e:
        pass
    return img


# โหลดภาพจาก URL และหมุนให้ถูกต้อง
def load_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = auto_orient_image(img)  # 🟢 หมุนตรงนี้ก่อน convert
    return img.convert("RGB")  # ค่อยแปลงเป็น RGB หลังจากหมุน


# โหลด MTCNN และ Resnet
mtcnn = MTCNN(image_size=160, margin=20, keep_all=True)
resnet = InceptionResnetV1(pretrained="vggface2").eval()


# เปลี่ยน PIL เป็น embedding (รองรับหลายใบหน้า)
def get_face_embeddings(pil_image):
    faces = mtcnn(pil_image)
    if faces is None:
        return None
    with torch.no_grad():
        embeddings = resnet(faces)
    return embeddings, faces


# เปรียบเทียบใบหน้าทุกใบในภาพที่ 2 กับภาพที่ 1
def compare_faces_mtcnn_facenet_multi(url1, url2, threshold=0.85):
    img1 = load_image(url1)
    img2 = load_image(url2)

    print("📸 ภาพที่ 1:")
    display(img1)
    print("📸 ภาพที่ 2:")
    display(img2)

    emb1_faces = mtcnn(img1)
    if emb1_faces is None or emb1_faces.shape[0] == 0:
        return "❌ ไม่พบใบหน้าในภาพที่ 1"
    emb1 = emb1_faces[0].unsqueeze(0)
    with torch.no_grad():
        emb1_vec = resnet(emb1)

    emb2_faces = get_face_embeddings(img2)
    if emb2_faces is None:
        return "❌ ไม่พบใบหน้าในภาพที่ 2"
    emb2, faces2 = emb2_faces
    if emb2 is None or emb2.shape[0] == 0:
        return "❌ ไม่พบใบหน้าในภาพที่ 2"

    distances = torch.norm(emb2 - emb1_vec, dim=1).cpu().numpy()

    for idx, (face_tensor, dist) in enumerate(zip(faces2, distances)):
        face_img = Image.fromarray(
            (face_tensor.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        )
        confidence = max(0.0, min(1.0, 1 - (dist / threshold)))  # confidence in [0,1]
        print(f"🔎 คนที่ {idx+1} | distance = {dist:.4f} | confidence = {confidence:.2%}")
        display(face_img)

    min_distance = distances.min()
    min_index = distances.argmin()

    if min_distance < threshold:
        return (
            f"✅ พบใบหน้าที่ใกล้เคียงที่สุด (คนที่ {min_index+1}) (distance = {min_distance:.4f})"
        )
    else:
        return f"❌ ไม่มีใบหน้าไหนตรงกับ default (min distance = {min_distance:.4f})"


# รูปพนักงาน
# url_default = "https://wms.advanceagro.net/WSVIS/api/Face/GetImage?CardID=688086"
url_default = "https://api-passport.advanceagro.net/qrapi/driver/1731056186658.jpg"

# ----------------------------------example for cannot detect face ----------------------------------
# mask
# url_new = "https://drive.google.com/uc?export=download&id=1oDlPFtFXdgQ8nyGGSi1K40Sn6AiuNIwJ"

# ----------------------------------example for detect successfully----------------------------------
# หันข้าง
# url_new = "https://drive.google.com/uc?export=download&id=1doQz85QDGwWO1lSfscVEZB1WCGKZahJz" #~1.0 + แต่ตั้งให้ 1 ไม่ได้เพราะ somehow บางคนต่ำกว่า 1

# can detect but not same person
# url_new = "https://drive.google.com/uc?export=download&id=1CWEzTjtH8ZjCdt80iujaJfrmRKpt7HyB"
# url_new = "https://wms.advanceagro.net/WSVIS/api/Face/GetImage?CardID=678069"

# 1people
# url_new = "https://drive.google.com/uc?export=download&id=1djZWka58x4sUX1uEJz2T_RB53Xj6k1jd"
# url_new = "https://drive.google.com/uc?export=download&id=16RpgIYZ1teAtJ2Ol4ZiaondmiAbQ6ryb"
# url_new = "https://drive.google.com/uc?export=download&id=1_cznadDWIyjdDvnahNsp--5UynDVuaxj"

# 2people
# url_new = "https://drive.google.com/uc?export=download&id=12N4vlF3shvRNMeyzW11LsCjXL0kyAoXA"
# url_new = "https://drive.google.com/uc?export=download&id=1vjZwPbo8pZqe9-9RMgoyJTXIwrPU-ZBE"
# url_new = "https://drive.google.com/uc?export=download&id=124bTxw7wv9Nj5GwVsHyWbxptt0O1pPga"
# url_new = "https://drive.google.com/uc?export=download&id=1ey2DzQ9SHetzoKyY8eJny4vXevNgzUGA"

# 3people
# url_new = "https://drive.google.com/uc?export=download&id=1U_ZCNcb8W-Y_e-qAv3lI1E6ix4vQF17a"

# group
# url_new = "https://drive.google.com/uc?export=download&id=1DIstBXIQ5TKy7um2T7-JBTP2Wk1MJtcl"

# Aws
url_new = "https://api-passport.advanceagro.net/awsapi/filename/1749632505960_20250611_3150100171382.jpg"

result = compare_faces_mtcnn_facenet_multi(url_default, url_new)
print(result)

In [ ]:
from ultralytics import YOLO
from PIL import Image
import requests
import io
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# โหลดโมเดล YOLOv8
# model = YOLO("models_trained/best_thai_car.pt")
# model = YOLO("models_trained/best_CarModels.pt")
model = YOLO("yolov8n")
ALLOWED_CLASSES = ["bicycle", "car", "motorcycle", "bus", "truck"]

# รายการ URL ของภาพs

image_urls = [
"https://drive.google.com/uc?export=download&id=1YA0GEAChL82XUbl-3f66Mt0xU03Shu1_",
"https://drive.google.com/uc?export=download&id=1kX057YJBP7_XaJrMwFpvjjy7jfggnqcV",
"https://drive.google.com/uc?export=download&id=1HlYBiE0-dUSGWDLbON2NEAEWJQUeERbj",
"https://drive.google.com/uc?export=download&id=1nFyIaRxislzVQYbV4FhwCJH98Hm84wF3",
"https://drive.google.com/uc?export=download&id=1cmeIrgDIJcuUrbHWYeHt51wzmtGW0ZJ_",
"https://drive.google.com/uc?export=download&id=1SwUfG9mtoEb3PSoZSviDa217IZeB-9Gr",
"https://drive.google.com/uc?export=download&id=1eD6XvJpc6_4sN_NWAR3LQKYL1lwo_9xo",
"https://drive.google.com/uc?export=download&id=1-Lay8p-1FIRtSChpIxkUJafYB47oO4KK",
"https://drive.google.com/uc?export=download&id=1JReVI-XEmEfXZyW8D1YrzLWrnfxRLIsG",
"https://drive.google.com/uc?export=download&id=1dugb8DpSBQSuYJ_nijPPPp_6rUUWX92p",
"https://drive.google.com/uc?export=download&id=1ZF-SBhNn0FvBFBidKkATupH2bVrA5ymI",
"https://drive.google.com/uc?export=download&id=13ohnqll-IjfzeWCdbJTarKj9vtifdwya",
"https://drive.google.com/uc?export=download&id=1muxEtmrmHH0rMQu4VadG00g6KDMXCzW5",
"https://drive.google.com/uc?export=download&id=168sKGSgz5RhldJERV7RRBCxokiQeZmMn",
"https://drive.google.com/uc?export=download&id=1lx-i_tRDzOTKGgKldG6punhXzbQXVw-A",
"https://drive.google.com/uc?export=download&id=13NYpvWs1D5PjIpV7pKFKhyVtlcFJM3DT",
"https://drive.google.com/uc?export=download&id=1X18kZkM0ioFz08J8q174HvwwUZcytcG2",
"https://drive.google.com/uc?export=download&id=1k4L_250gyjOSQ5QjVTyYAJqRLiLw2pbR",
"https://drive.google.com/uc?export=download&id=1I2bw1ci2kUO-qAOnqZz1ndKlR4PaUL_o",
"https://drive.google.com/uc?export=download&id=1PhhR1s4q0VcIZnCT_7MHSm4MBFaidur8",
"https://drive.google.com/uc?export=download&id=1kWKgps6UB-PdsUnoNcyAojA6EObzgeRn",
"https://drive.google.com/uc?export=download&id=1MGnWyEmvTmcXpoMYd-iWtCit1DVEEzPf",
"https://drive.google.com/uc?export=download&id=1qtvw9SOBpbr6X8El5x30XC-gNoxly3Aw"
]


for idx, url in enumerate(image_urls):
    print(f"\n🖼️ ตรวจภาพที่ {idx + 1} ...")
    try:
        # โหลดภาพจาก URL
        response = requests.get(url)
        image = Image.open(io.BytesIO(response.content)).convert("RGB")

        # รันโมเดล
        results = model(image)
        predictions = []

        # แปลงผลลัพธ์เป็น list ของ predictions
        for box in results[0].boxes:
            class_id = int(box.cls[0].cpu())
            class_name = model.names[class_id]
            confidence = float(box.conf[0].cpu()) * 100

            if class_name not in ALLOWED_CLASSES:
                continue

            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            area = (x2 - x1) * (y2 - y1)

            predictions.append(
                {
                    "type": class_name,
                    "confidence": confidence,
                    "area": area,
                    "box": (x1, y1, x2, y2),
                }
            )

        if not predictions:
            print("⚠️ No known vehicle type detected.")
            continue

        # วิเคราะห์: หาพื้นที่ใหญ่สุด แล้วกรองเฉพาะที่ ≥ 90% ของ max
        max_area = max(p["area"] for p in predictions)
        filtered = [p for p in predictions if p["area"] >= 0.9 * max_area]

        # เลือก top prediction
        top_pred = (
            max(filtered, key=lambda x: x["confidence"])
            if filtered
            else max(predictions, key=lambda x: x["area"])
        )

        # แสดงภาพพร้อมกล่อง
        fig, ax = plt.subplots(1, figsize=(10, 8))
        ax.imshow(image)

        for i, p in enumerate(predictions):
            x1, y1, x2, y2 = p["box"]
            is_top = p == top_pred

            # กล่องสี่เหลี่ยม
            rect = patches.Rectangle(
                (x1, y1),
                x2 - x1,
                y2 - y1,
                linewidth=2,
                edgecolor="red" if is_top else "lime",
                facecolor="none",
            )
            ax.add_patch(rect)

            # ข้อความ label สลับบนล่าง
            label_y = y1 - 10 if i % 2 == 0 else y2 + 10
            label = f"{p['type']} {p['confidence']:.1f}%"
            ax.text(
                x1,
                label_y,
                label,
                color="white",
                backgroundcolor="red" if is_top else "green",
                fontsize=12,
            )

        plt.axis("off")
        plt.title(f"Image {idx + 1}")
        plt.show()

        # รายงานผลทั้งหมด
        print("🔍 all_predictions:")
        for p in predictions:
            area_diff_pct = (1 - p["area"] / max_area) * 100
            print(
                f"   - {p['type']} ({p['confidence']:.1f}%) | "
                f"Area: {int(p['area'])} | ΔArea: {area_diff_pct:.1f}%"
            )

        print(
            f"🏆 Top prediction (area+confidence) for image {idx + 1}: "
            f"{top_pred['type']} ({top_pred['confidence']:.1f}%)"
        )

    except Exception as e:
        print(f"❌ Error loading image {idx + 1}: {e}")

In [ ]:
from ultralytics import YOLO
from PIL import Image
import requests
import io
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# โหลดโมเดล YOLOv8
model = YOLO("yolov8n.pt")
ALLOWED_CLASSES = ["bicycle", "car", "motorcycle", "bus", "truck"]

# รายการ URL ของภาพ
image_urls = [
# "https://drive.google.com/uc?export=download&id=1YA0GEAChL82XUbl-3f66Mt0xU03Shu1_",
# "https://drive.google.com/uc?export=download&id=1kX057YJBP7_XaJrMwFpvjjy7jfggnqcV",d
# "https://drive.google.com/uc?export=download&id=1HlYBiE0-dUSGWDLbON2NEAEWJQUeERbj",
# "https://drive.google.com/uc?export=download&id=1nFyIaRxislzVQYbV4FhwCJH98Hm84wF3",
# "https://drive.google.com/uc?export=download&id=1cmeIrgDIJcuUrbHWYeHt51wzmtGW0ZJ_",
"https://drive.google.com/uc?export=download&id=1SwUfG9mtoEb3PSoZSviDa217IZeB-9Gr",
# "https://drive.google.com/uc?export=download&id=1eD6XvJpc6_4sN_NWAR3LQKYL1lwo_9xo",
# "https://drive.google.com/uc?export=download&id=1-Lay8p-1FIRtSChpIxkUJafYB47oO4KK",
# "https://drive.google.com/uc?export=download&id=1JReVI-XEmEfXZyW8D1YrzLWrnfxRLIsG",
# "https://drive.google.com/uc?export=download&id=1dugb8DpSBQSuYJ_nijPPPp_6rUUWX92p",
# "https://drive.google.com/uc?export=download&id=1ZF-SBhNn0FvBFBidKkATupH2bVrA5ymI",
# "https://drive.google.com/uc?export=download&id=13ohnqll-IjfzeWCdbJTarKj9vtifdwya",
# "https://drive.google.com/uc?export=download&id=1muxEtmrmHH0rMQu4VadG00g6KDMXCzW5",
# "https://drive.google.com/uc?export=download&id=168sKGSgz5RhldJERV7RRBCxokiQeZmMn",
# "https://drive.google.com/uc?export=download&id=1lx-i_tRDzOTKGgKldG6punhXzbQXVw-A",
# "https://drive.google.com/uc?export=download&id=13NYpvWs1D5PjIpV7pKFKhyVtlcFJM3DT",
# "https://drive.google.com/uc?export=download&id=1X18kZkM0ioFz08J8q174HvwwUZcytcG2",
# "https://drive.google.com/uc?export=download&id=1k4L_250gyjOSQ5QjVTyYAJqRLiLw2pbR",
# "https://drive.google.com/uc?export=download&id=1I2bw1ci2kUO-qAOnqZz1ndKlR4PaUL_o",
# "https://drive.google.com/uc?export=download&id=1PhhR1s4q0VcIZnCT_7MHSm4MBFaidur8",
# "https://drive.google.com/uc?export=download&id=1kWKgps6UB-PdsUnoNcyAojA6EObzgeRn",
# "https://drive.google.com/uc?export=download&id=1MGnWyEmvTmcXpoMYd-iWtCit1DVEEzPf",
# "https://drive.google.com/uc?export=download&id=1qtvw9SOBpbr6X8El5x30XC-gNoxly3Aw"
]


for idx, url in enumerate(image_urls):
    print(f"\n🖼️ ตรวจภาพที่ {idx + 1} ...")
    try:
        # โหลดภาพจาก URL
        response = requests.get(url)
        image = Image.open(io.BytesIO(response.content)).convert("RGB")

        # รันโมเดล
        results = model(image)
        predictions = []

        # แปลงผลลัพธ์เป็น list ของ predictions
        for box in results[0].boxes:
            class_id = int(box.cls[0].cpu())
            class_name = model.names[class_id]
            confidence = float(box.conf[0].cpu()) * 100



            if class_name not in ALLOWED_CLASSES:
                continue

            x1, y1, x2, y2 = map(float, box.xyxy[0].cpu())
            area = (x2 - x1) * (y2 - y1)

            predictions.append({
                "type": class_name,
                "confidence": confidence,
                "area": area,
                "box": (x1, y1, x2, y2)
            })
        if not predictions:
            print("⚠️ No known vehicle type detected.")
            continue

        # วิเคราะห์: หาพื้นที่ใหญ่สุด แล้วกรองเฉพาะที่ ≥ 90% ของ max
        max_area = max(p["area"] for p in predictions)
        filtered = [p for p in predictions if p["area"] >= 0.9 * max_area]

        # เลือก top prediction
        top_pred = (
            max(filtered, key=lambda x: x["confidence"])
            if filtered else
            max(predictions, key=lambda x: x["area"])
        )

        # แสดงภาพพร้อมกล่อง
        fig, ax = plt.subplots(1, figsize=(10, 8))
        ax.imshow(image)

        for i, p in enumerate(predictions):
            x1, y1, x2, y2 = p["box"]
            is_top = p == top_pred

            # กล่องสี่เหลี่ยม
            rect = patches.Rectangle(
                (x1, y1), x2 - x1, y2 - y1,
                linewidth=2,
                edgecolor='red' if is_top else 'lime',
                facecolor='none'
            )
            ax.add_patch(rect)

            # ข้อความ label สลับบนล่าง
            label_y = y1 - 10 if i % 2 == 0 else y2 + 10
            label = f"{p['type']} {p['confidence']:.1f}%"
            ax.text(
                x1, label_y, label,
                color='white',
                backgroundcolor='red' if is_top else 'green',
                fontsize=12
            )

        plt.axis('off')
        plt.title(f"Image {idx + 1}")
        plt.show()

        # รายงานผลทั้งหมด
        print("🔍 all_predictions:")
        for p in predictions:
            area_diff_pct = (1 - p["area"] / max_area) * 100
            print(f"   - {p['type']} ({p['confidence']:.1f}%) | "
                  f"Area: {int(p['area'])} | ΔArea: {area_diff_pct:.1f}%")

        print(f"🏆 Top prediction (area+confidence) for image {idx + 1}: "
              f"{top_pred['type']} ({top_pred['confidence']:.1f}%)")

    except Exception as e:
        print(f"❌ Error loading image {idx + 1}: {e}")

In [ ]:
raw_text = """
https://drive.google.com/file/d/1kX057YJBP7_XaJrMwFpvjjy7jfggnqcV/view?usp=drive_link
https://drive.google.com/file/d/1HlYBiE0-dUSGWDLbON2NEAEWJQUeERbj/view?usp=drive_link
https://drive.google.com/file/d/1nFyIaRxislzVQYbV4FhwCJH98Hm84wF3/view?usp=drive_link
https://drive.google.com/file/d/1cmeIrgDIJcuUrbHWYeHt51wzmtGW0ZJ_/view?usp=drive_link
"""

# แปลงเป็นลิสต์และจัดรูปแบบ
urls = [f'"{line.strip()}"' for line in raw_text.strip().splitlines() if line.strip()]

# พิมพ์เป็น list พร้อม ,
print("urls = [\n" + ",\n".join(urls) + "\n]")

# แปลงเป็นลิสต์ทีละบรรทัด
urls = [line.strip() for line in raw_text.strip().splitlines() if line.strip()]

# แปลงลิงก์ให้เป็น direct link
converted_urls = []
for url in urls:
    try:
        file_id = url.split("/d/")[1].split("/")[0]
        direct_url = f"https://drive.google.com/uc?export=download&id={file_id}"
        converted_urls.append(f'"{direct_url}",')
    except:
        print(f"❌ Invalid URL: {url}")

print("\nConverted URLs:\n")
for u in converted_urls:
    print(u)

# image_urls = [
# "https://drive.google.com/uc?export=download&id=1YA0GEAChL82XUbl-3f66Mt0xU03Shu1_",
# "https://drive.google.com/uc?export=download&id=1kX057YJBP7_XaJrMwFpvjjy7jfggnqcV",
# "https://drive.google.com/uc?export=download&id=1HlYBiE0-dUSGWDLbON2NEAEWJQUeERbj",
# "https://drive.google.com/uc?export=download&id=1nFyIaRxislzVQYbV4FhwCJH98Hm84wF3",
# "https://drive.google.com/uc?export=download&id=1cmeIrgDIJcuUrbHWYeHt51wzmtGW0ZJ_",
# "https://drive.google.com/uc?export=download&id=1SwUfG9mtoEb3PSoZSviDa217IZeB-9Gr",
# "https://drive.google.com/uc?export=download&id=1eD6XvJpc6_4sN_NWAR3LQKYL1lwo_9xo",
# "https://drive.google.com/uc?export=download&id=1-Lay8p-1FIRtSChpIxkUJafYB47oO4KK",
# "https://drive.google.com/uc?export=download&id=1JReVI-XEmEfXZyW8D1YrzLWrnfxRLIsG",
# "https://drive.google.com/uc?export=download&id=1dugb8DpSBQSuYJ_nijPPPp_6rUUWX92p",
# "https://drive.google.com/uc?export=download&id=1ZF-SBhNn0FvBFBidKkATupH2bVrA5ymI",
# "https://drive.google.com/uc?export=download&id=13ohnqll-IjfzeWCdbJTarKj9vtifdwya",
# "https://drive.google.com/uc?export=download&id=1muxEtmrmHH0rMQu4VadG00g6KDMXCzW5",
# "https://drive.google.com/uc?export=download&id=168sKGSgz5RhldJERV7RRBCxokiQeZmMn",
# "https://drive.google.com/uc?export=download&id=1lx-i_tRDzOTKGgKldG6punhXzbQXVw-A",
# "https://drive.google.com/uc?export=download&id=13NYpvWs1D5PjIpV7pKFKhyVtlcFJM3DT",
# "https://drive.google.com/uc?export=download&id=1X18kZkM0ioFz08J8q174HvwwUZcytcG2",
# "https://drive.google.com/uc?export=download&id=1k4L_250gyjOSQ5QjVTyYAJqRLiLw2pbR",
# "https://drive.google.com/uc?export=download&id=1I2bw1ci2kUO-qAOnqZz1ndKlR4PaUL_o",
# "https://drive.google.com/uc?export=download&id=1PhhR1s4q0VcIZnCT_7MHSm4MBFaidur8",
# "https://drive.google.com/uc?export=download&id=1kWKgps6UB-PdsUnoNcyAojA6EObzgeRn",
# "https://drive.google.com/uc?export=download&id=1MGnWyEmvTmcXpoMYd-iWtCit1DVEEzPf",
# "https://drive.google.com/uc?export=download&id=1qtvw9SOBpbr6X8El5x30XC-gNoxly3Aw"
# ]



In [ ]:
import os
import shutil

# YOLO target class: ['bicycle', 'car', 'motorcycle', 'bus', 'truck']
#                  =>     0      1           2       3       4
class_mapping = {
    0: 4,  # full_trailer     → truck
    1: 3,  # heavy_bus        → bus
    2: 4,  # heavy_truck      → truck
    3: 3,  # light_bus        → bus
    4: 4,  # light_truck      → truck
    5: 3,  # medium_bus       → bus
    6: 4,  # medium_truck     → truck
    7: 2,  # motorcycle       → motorcycle
    8: 1,  # passenger_car    → car
    9: 4,  # semi_trailer     → truck
    10: 1, # van              → car
}

# ✅ ฟังก์ชันรีแมป label
def remap_labels(src_dir, dst_dir, class_mapping):
    os.makedirs(dst_dir, exist_ok=True)
    for fname in os.listdir(src_dir):
        if not fname.endswith(".txt"):
            continue
        src_file = os.path.join(src_dir, fname)
        dst_file = os.path.join(dst_dir, fname)

        with open(src_file, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if not parts:
                continue
            old_cls = int(parts[0])
            if old_cls in class_mapping:
                new_cls = class_mapping[old_cls]
                parts[0] = str(new_cls)
                new_lines.append(" ".join(parts))

        if new_lines:
            with open(dst_file, "w") as f:
                f.write("\n".join(new_lines))

# ✅ ฟังก์ชันคัดลอกภาพ
def copy_images(src_dir, dst_dir):
    os.makedirs(dst_dir, exist_ok=True)
    for fname in os.listdir(src_dir):
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            shutil.copy(os.path.join(src_dir, fname), os.path.join(dst_dir, fname))

# ✅ ต้นทาง
base_src = r"C:\Users\User\Downloads\DoubleA\python\Thailand-vehicles.v25i.yolov8"
src_train_labels = os.path.join(base_src, "train", "labels")
src_train_images = os.path.join(base_src, "train", "images")

# ✅ ปลายทาง
base_dst = r"C:\Users\User\Downloads\DoubleA\python\dataset"
dst_train_labels = os.path.join(base_dst, "labels", "train")
dst_train_images = os.path.join(base_dst, "images", "train")

# ✅ เรียกใช้
remap_labels(src_train_labels, dst_train_labels, class_mapping)
copy_images(src_train_images, dst_train_images)

print("✅ Remap และคัดลอกเรียบร้อยแล้ว พร้อมใช้เทรน YOLO ได้เลย!")


In [ ]:
import os
import random
import shutil

# YOLO target class: ['bicycle', 'car', 'motorcycle', 'bus', 'truck']
#                  =>     0      1           2       3       4
class_mapping = {
    0: 4,  # full_trailer     → truck
    1: 3,  # heavy_bus        → bus
    2: 4,  # heavy_truck      → truck
    3: 3,  # light_bus        → bus
    4: 4,  # light_truck      → truck
    5: 3,  # medium_bus       → bus
    6: 4,  # medium_truck     → truck
    7: 2,  # motorcycle       → motorcycle
    8: 1,  # passenger_car    → car
    9: 4,  # semi_trailer     → truck
    10: 1, # van              → car
}

# ✅ ฟังก์ชันแปลง label
def remap_and_save_label(src_file, dst_file, class_mapping):
    with open(src_file, "r") as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        parts = line.strip().split()
        if not parts:
            continue
        old_cls = int(parts[0])
        if old_cls in class_mapping:
            parts[0] = str(class_mapping[old_cls])
            new_lines.append(" ".join(parts))

    if new_lines:
        with open(dst_file, "w") as f:
            f.write("\n".join(new_lines))

# ✅ แบ่ง train → train + val พร้อม remap labels
def split_and_remap(src_img_dir, src_lbl_dir, dst_base_dir, train_ratio=0.8):
    images = [f for f in os.listdir(src_img_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    random.shuffle(images)

    split_idx = int(len(images) * train_ratio)
    train_imgs = images[:split_idx]
    val_imgs   = images[split_idx:]

    for split, file_list in [("train", train_imgs), ("val", val_imgs)]:
        img_dst = os.path.join(dst_base_dir, "images", split)
        lbl_dst = os.path.join(dst_base_dir, "labels", split)
        os.makedirs(img_dst, exist_ok=True)
        os.makedirs(lbl_dst, exist_ok=True)

        for img_name in file_list:
            lbl_name = os.path.splitext(img_name)[0] + ".txt"
            shutil.copy(os.path.join(src_img_dir, img_name), os.path.join(img_dst, img_name))
            remap_and_save_label(os.path.join(src_lbl_dir, lbl_name), os.path.join(lbl_dst, lbl_name), class_mapping)

# === เรียกใช้งาน ===
base_src = r"C:\Users\User\Downloads\DoubleA\python\Thailand-vehicles.v25i.yolov8\train"
base_dst = r"C:\Users\User\Downloads\DoubleA\python\dataset"

split_and_remap(
    src_img_dir=os.path.join(base_src, "images"),
    src_lbl_dir=os.path.join(base_src, "labels"),
    dst_base_dir=base_dst,
    train_ratio=0.8  # ปรับ % ได้ตามต้องการ เช่น 0.9 = 90% train, 10% val
)

print("✅ แบ่ง train/val และ remap labels เรียบร้อย พร้อมเทรน YOLO!")
